### EDA
#### Step 1 : 分析各產業別之間的異常報酬率

In [1]:
import pandas as pd
import seaborn as sns
import bokeh as bok
df_fs_return=pd.read_csv("上市櫃公司_plus異常報酬率.csv")

In [16]:
import numpy as np
df_fs_return_columns=df_fs_return.columns[51:]
return_industry=df_fs_return.groupby(by="TSE產業名稱")[df_fs_return_columns].mean()
print("資料集大小 :",return_industry.shape)
print(f"總計有{len(np.unique(df_fs_return['TSE產業名稱'].values.flatten()))}種產業類別")

資料集大小 : (35, 132)
總計有35種產業類別


### 動態圖表 : 完全由Gemini pro 3 製作

In [53]:
import pandas as pd
import plotly.express as px
import numpy as np

# 1. 讀取資料
df = pd.read_csv('return_industry.csv')

# 清除可能存在的雜訊欄位
if 'Unnamed: 182' in df.columns:
    df = df.drop(columns=['Unnamed: 182'])

# 2. 資料轉置：從 Wide Format 轉為 Long Format
# id_vars 是固定欄位 (產業名稱)，其餘欄位都是時間點
id_col = df.columns[0]  # 'TSE產業名稱'
time_cols = df.columns[1:]  # '-10', '-9', ... 等時間欄位

df_long = df.melt(id_vars=[id_col], value_vars=time_cols, var_name='Day', value_name='Return')

# 3. 資料清理：將天數轉為整數並排序
df_long['Day'] = df_long['Day'].astype(int)
df_long = df_long.sort_values(by=[id_col, 'Day'])

# 4. 計算累積報酬 (Trend Growth 的核心)
df_long['Cumulative_Return'] = df_long.groupby(id_col)['Return'].cumsum()

# 5. 製作動畫專用的資料結構 (Data Inflation)
# 為了讓線條有「生長」的效果，每一個 Frame (時間點) 必須包含「過去所有的資料」
frames_data = []
unique_days = sorted(df_long['Day'].unique())

for day in unique_days:
    # 篩選出「目前時間點之前」的所有資料
    subset = df_long[df_long['Day'] <= day].copy()
    subset['Frame'] = day  # 設定動畫的時間軸標記
    frames_data.append(subset)

# 合併所有 Frame 的資料 (這會使資料量變大，但能確保動畫流暢)
animated_df = pd.concat(frames_data)

distinct_colors = px.colors.qualitative.Alphabet + px.colors.qualitative.Dark24

# 6. 繪製動態折線圖 (加入 color_discrete_sequence)
fig = px.line(
    animated_df,
    x="Day",
    y="Cumulative_Return",
    color=id_col,
    animation_frame="Frame",
    title="各產業累積異常報酬率動態趨勢圖",
    labels={"Cumulative_Return": "Cumulative Abnormal Return (%)", "Day": "Event Day"},
    range_x=[min(unique_days), max(unique_days)],
    range_y=[df_long['Cumulative_Return'].min()*1.1, df_long['Cumulative_Return'].max()*1.1],
    
    # ★ 關鍵修改在這裡：指定自訂的超長色票
    color_discrete_sequence=distinct_colors
)

# 優化版面 (建議把 Legend 字體縮小，以免遮住圖表)
fig.update_layout(
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    hovermode="x unified",
    legend=dict(
        font=dict(size=9), # 縮小圖例字體
        itemwidth=30       # 縮窄圖例寬度
    ),
    updatemenus=[dict(type='buttons', showactive=False,
                                buttons=[dict(label='Play',
                                            method='animate',
                                            args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True)])])]
)

fig.write_html("trend_growth_animation_v2.html")
print("已生成顏色改良版：trend_growth_animation_v2.html")

已生成顏色改良版：trend_growth_animation_v2.html
